Для предложенных данных установить наличие мультиколлинеарности. Для этого:

1. Первоначально провести обычный регрессионный анализ. Проанализировать полученные результаты. Выписать коэффициент детерминации;
2. Рассчитать корреляционную матрицу всех переменных. Установить зависимость между факторами;
3. Построить регрессию одного из факторов на остальные факторы (Y в этой модели не рассматривать). Сравнить коэффициент детерминации этой модели, с коэффициентом детерминации в регрессионной модели Y на все факторы;
4. Дополнительно в пакете R вычислить число обусловленности матрицы, как отношение максимального и минимального характеристических чисел информационной матрицы J.
5. Устранить мультиколлинеарность методом пошаговой регрессии (Stepwise regression), оставив только значимые факторы.


In [1]:
data <- read.csv(file = "../Data/Cars.csv", header = TRUE, dec = ",")[,-1] # Удалил навзание
head(data)

,CYL,POW,LEN,WID,WT,MAX.SP,PRICE
,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1350,79,393,161,870,165,30570
2,1588,85,468,177,1110,160,39990
3,1294,68,424,168,1050,152,29600
4,1222,59,412,161,930,151,28250
5,1585,98,439,164,1105,165,34900
6,1297,82,429,169,1080,160,35480


In [2]:
regModel <- lm(data = data[,-7])

In [3]:
summary(regModel)


Call:
lm(data = data[, -7])

Residuals:
    Min      1Q  Median      3Q     Max 
-365.78 -136.99    3.42  108.11  353.36 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)
(Intercept) -2128.407   2135.049  -0.997    0.338
POW             3.374      9.043   0.373    0.716
LEN            -1.401      6.736  -0.208    0.839
WID             8.096     21.302   0.380    0.711
WT              1.420      1.213   1.170    0.265
MAX.SP          7.588     11.350   0.669    0.516

Residual standard error: 229.2 on 12 degrees of freedom
Multiple R-squared:  0.7349,	Adjusted R-squared:  0.6244 
F-statistic: 6.653 on 5 and 12 DF,  p-value: 0.003461


In [4]:
# Информационная матрица
J <- t( as.matrix(data))%*%as.matrix(data)
J
det(J) # != 0 

,CYL,POW,LEN,WID,WT,MAX.SP,PRICE
CYL,50299050,2588214,12830473,4916272,32372208,4699934,1029914660
POW,2588214,135921,665132,254792,1679370,244608,53841705
LEN,12830473,665132,3390909,1302196,8462819,1237213,268129570
WID,4916272,254792,1302196,500480,3245369,475352,102800340
WT,32372208,1679370,8462819,3245369,21268741,3087096,674851850
MAX.SP,4699934,244608,1237213,475352,3087096,453439,98106915
PRICE,1029914660,53841705,268129570,102800340,674851850,98106915,21736748625


[1] 2.351728e+34

In [5]:
eigen(J) # Собств значения

eigen() decomposition
$values
[1] 2.181087e+10 1.638601e+06 2.593006e+05 2.224138e+04 2.400678e+03
[6] 3.884322e+02 1.223564e+02

$vectors
             [,1]        [,2]        [,3]         [,4]         [,5]
[1,] -0.047304592  0.94610400  0.31371552 -0.056285929 -0.032604320
[2,] -0.002472870  0.02283956  0.01856490  0.029629757  0.780704739
[3,] -0.012314862  0.10386548 -0.45907364 -0.691812947 -0.205695064
[4,] -0.004721490  0.03846250 -0.18696119 -0.342750167 -0.067937188
[5,] -0.030995192  0.29573534 -0.79963569  0.513863658  0.047500566
[6,] -0.004505926  0.03918137 -0.12580757 -0.368528022  0.583311376
[7,] -0.998299157 -0.05570991  0.01703078 -0.001542181 -0.001637809
              [,6]          [,7]
[1,] -0.0009396090 -0.0001683084
[2,]  0.5173912341 -0.3479311460
[3,]  0.4773643109  0.1718121162
[4,] -0.4209478542 -0.8150175585
[5,] -0.0747696359  0.0161249464
[6,] -0.5671340970  0.4300125867
[7,] -0.0002536355  0.0001634819


In [6]:
sqrt(max(eigen(J)$values)/min(eigen(J)$values)) # Число обусловленностей
                                                # Анализ главных компонент

[1] 13351.29

In [7]:
summary(lm(PRICE ~ 1, data = data))


Call:
lm(formula = PRICE ~ 1, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-12058.6  -4316.1   -813.6   4298.9  13541.4 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)    34159       1549   22.05 6.03e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 6572 on 17 degrees of freedom


In [8]:
for (i in names(data)){
    cat( i )
    print(summary(lm(as.formula(paste(i, "~ .")), data=data)))
    cat('\n\n')
}

#Интересно, что самый низкий коэффицент детерминации на целевой переменной: цена

CYL
Call:
lm(formula = as.formula(paste(i, "~ .")), data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-347.92 -125.98  -19.56   79.40  373.48 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)
(Intercept) -2.136e+03  2.191e+03  -0.975    0.350
POW          6.072e+00  1.022e+01   0.594    0.564
LEN         -1.502e+00  6.913e+00  -0.217    0.832
WID          9.896e+00  2.204e+01   0.449    0.662
WT           1.496e+00  1.251e+00   1.196    0.257
MAX.SP       6.213e+00  1.185e+01   0.524    0.610
PRICE       -9.981e-03  1.580e-02  -0.631    0.541

Residual standard error: 235.1 on 11 degrees of freedom
Multiple R-squared:  0.7442,	Adjusted R-squared:  0.6046 
F-statistic: 5.333 on 6 and 11 DF,  p-value: 0.008311



POW
Call:
lm(formula = as.formula(paste(i, "~ .")), data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.3297 -3.6945  0.2192  3.4855 13.0372 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.97

In [9]:
# Пошаговая регрессия. 
# Частнная корреляция самая высокая PRICE c POW

summary(lm(PRICE ~ POW , data = data))


Call:
lm(formula = PRICE ~ POW, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-7779.8 -2483.9   256.4  1993.9  6479.2 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 12363.65    4215.92   2.933  0.00976 ** 
POW           257.59      48.52   5.309 7.05e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4076 on 16 degrees of freedom
Multiple R-squared:  0.6379,	Adjusted R-squared:  0.6153 
F-statistic: 28.19 on 1 and 16 DF,  p-value: 7.052e-05


In [10]:
RSS <- sum(residuals(lm(PRICE ~ POW, data = data))^2)

In [11]:
RSS_j <- numeric()

for (i in names(data)){
    RSS_j[i] <- sum((lm(as.formula(paste("PRICE ~ POW + ", i)), data=data)$residuals)^2)
}

F_j <- as.numeric(RSS) - as.numeric(RSS_j)

F_j <- F_j / as.numeric(RSS_j)

F_j <- F_j * (18-3)

F_j

#WT макс значение

Warning message in model.matrix.default(mt, mf, contrasts):
“the response appeared on the right-hand side and was dropped”
Warning message in model.matrix.default(mt, mf, contrasts):
“problem with term 2 in model.matrix: no columns are assigned”


[1] 0.0006072842 0.0000000000 1.3244364283 1.0376732328 2.3313090582
[6] 1.3504179416 0.0000000000

In [12]:
summary(lm(PRICE ~ POW + WT , data = data))
#WT Выводится как незначимый, но коэффицент детерминации вырос


Call:
lm(formula = PRICE ~ POW + WT, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-7148.2 -1874.6   296.3  2012.7  5263.7 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept)  1775.60    8030.95   0.221   0.8280  
POW           172.97      72.42   2.388   0.0305 *
WT             16.45      10.77   1.527   0.1476  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3916 on 15 degrees of freedom
Multiple R-squared:  0.6866,	Adjusted R-squared:  0.6448 
F-statistic: 16.43 on 2 and 15 DF,  p-value: 0.0001661


In [13]:
RSS <- sum(residuals(lm(PRICE ~ POW + WT , data = data))^2)
RSS
RSS_j <- numeric()

for (i in names(data)){
    RSS_j[i] <- sum((lm(as.formula(paste("PRICE ~ POW + ", i)), data=data)$residuals)^2)
}

F_j <- as.numeric(RSS) - as.numeric(RSS_j)

F_j <- F_j / as.numeric(RSS_j)

F_j <- F_j * (18-4)

F_j

# Хватит добавлять факторы.
# Првоерить отсутсвие свободного члена.

[1] 230064636

Warning message in model.matrix.default(mt, mf, contrasts):
“the response appeared on the right-hand side and was dropped”
Warning message in model.matrix.default(mt, mf, contrasts):
“problem with term 2 in model.matrix: no columns are assigned”


[1] -1.8827098 -1.8832003 -0.8133383 -1.0449817  0.0000000 -0.7923507 -1.8832003

In [14]:
summary(lm(PRICE ~ POW + WT -1  , data = data))


Call:
lm(formula = PRICE ~ POW + WT - 1, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-6961.7 -1818.2   262.7  2143.7  5212.9 

Coefficients:
    Estimate Std. Error t value Pr(>|t|)   
POW  167.448     65.930   2.540  0.02185 * 
WT    18.508      5.271   3.512  0.00289 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3798 on 16 degrees of freedom
Multiple R-squared:  0.9894,	Adjusted R-squared:  0.9881 
F-statistic: 745.4 on 2 and 16 DF,  p-value: < 2.2e-16


In [15]:
RSS <- sum(residuals(lm(PRICE ~ POW + WT -1 , data = data))^2)
RSS
RSS_j <- numeric()

for (i in names(data)){
    RSS_j[i] <- sum((lm(as.formula(paste("PRICE ~ POW + ", i, " -1")), data=data)$residuals)^2)
}

F_j <- as.numeric(RSS) - as.numeric(RSS_j)

F_j <- F_j / as.numeric(RSS_j)

F_j <- F_j * (18-2)

F_j

[1] 230814385

Warning message in model.matrix.default(mt, mf, contrasts):
“the response appeared on the right-hand side and was dropped”
Warning message in model.matrix.default(mt, mf, contrasts):
“problem with term 2 in model.matrix: no columns are assigned”


[1] -6.464822 -6.964040 -1.199184 -1.550056  0.000000 -3.598375 -6.964040

In [16]:
#Лучшая модель
summary(lm(PRICE ~ POW + WT -1  , data = data))


Call:
lm(formula = PRICE ~ POW + WT - 1, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-6961.7 -1818.2   262.7  2143.7  5212.9 

Coefficients:
    Estimate Std. Error t value Pr(>|t|)   
POW  167.448     65.930   2.540  0.02185 * 
WT    18.508      5.271   3.512  0.00289 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3798 on 16 degrees of freedom
Multiple R-squared:  0.9894,	Adjusted R-squared:  0.9881 
F-statistic: 745.4 on 2 and 16 DF,  p-value: < 2.2e-16


In [17]:
regModel0 <- lm(PRICE ~ - 1, data = data)
regModelfull <- lm(PRICE ~ ., data = data)
stepForward <- step(regModel0, scope=list(lower=regModel0, upper=regModelfull), direction="forward")

Start:  AIC=376.41
PRICE ~ -1

         Df  Sum of Sq        RSS    AIC
+ WT      1 2.1413e+10 3.2387e+08 302.70
+ POW     1 2.1328e+10 4.0870e+08 306.89
+ MAX.SP  1 2.1227e+10 5.1015e+08 310.88
+ LEN     1 2.1202e+10 5.3492e+08 311.73
+ WID     1 2.1116e+10 6.2120e+08 314.42
+ CYL     1 2.1088e+10 6.4839e+08 315.19
<none>                 2.1737e+10 376.41

Step:  AIC=302.7
PRICE ~ WT - 1

         Df Sum of Sq       RSS    AIC
+ POW     1  93053269 230814385 298.60
<none>                323867653 302.70
+ CYL     1   7379268 316488385 304.28
+ LEN     1   6574914 317292740 304.33
+ WID     1   5768263 318099390 304.38
+ MAX.SP  1   4435340 319432314 304.45

Step:  AIC=298.6
PRICE ~ WT + POW - 1

         Df Sum of Sq       RSS    AIC
<none>                230814385 298.60
+ CYL     1   9630861 221183524 299.83
+ WID     1   1064465 229749919 300.52
+ LEN     1    738112 230076272 300.54
+ MAX.SP  1     25338 230789047 300.60


In [18]:
class(stepForward) # Он возращает лучшуб линеную модель.

[1] "lm"

In [19]:
summary(stepForward) # Как и у меня


Call:
lm(formula = PRICE ~ WT + POW - 1, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-6961.7 -1818.2   262.7  2143.7  5212.9 

Coefficients:
    Estimate Std. Error t value Pr(>|t|)   
WT    18.508      5.271   3.512  0.00289 **
POW  167.448     65.930   2.540  0.02185 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3798 on 16 degrees of freedom
Multiple R-squared:  0.9894,	Adjusted R-squared:  0.9881 
F-statistic: 745.4 on 2 and 16 DF,  p-value: < 2.2e-16
